In [98]:
import numpy as np
import random

Seed para reproducibilidade

In [ ]:
random.seed(323452345)

In [100]:
!git clone https://github.com/jesusdourado/rna-atividade2.1-data.git

fatal: destination path 'rna-atividade2.1-data' already exists and is not an empty directory.


In [102]:
exemplos = np.fromfile('rna-atividade2.1-data/data/dataAll.txt').reshape(-1,3)

In [ ]:
class Perceptron:

    def __init__(self, tamanho, bias = 0, taxa_aprendizado= 0.1) -> None:
        self.bias = bias
        self.taxa_aprendizado = taxa_aprendizado
        self.pesos = np.zeros(tamanho + 1)
        self.pesos[0] = random.uniform(-0.5,0.5)
        self.pesos[1] = random.uniform(-0.5,0.5)
        self.pesos[2] = random.uniform(-0.5,0.5)
        self.entradas = np.empty(tamanho + 1)
        self.entradas[0] = self.bias
        self.erro = 0
        self.atualiza_pesos_contador = 0 


    def treinar(self, d):
        epoca = 0
        continua = True
        while continua:
            epoca += 1
            print("Época: ", epoca)
            contem_erro = False
            qtd_ajustes_pesos_epoca = 0

            for i in range(d.shape[0]):

                self.entradas[1] = d[i][0]
                self.entradas[2] = d[i][1]

                y = self.predicao()
                self.calcula_erro(y, d[i][2])

                if self.erro != 0:
                    self.atualiza_pesos()
                    qtd_ajustes_pesos_epoca += 1
                    contem_erro = True      

            print("Total ajustes de pesos da época: ", qtd_ajustes_pesos_epoca)

            if not contem_erro:
                continua = False
        
        print("Número total de ajustes dos pesos: ", self.atualiza_pesos_contador)
        print("Número de Épocas: ", epoca)

    def funcao_de_ativacao(self, x):
        return 1 if x >= 0 else 0

    def predicao(self):
        y = self.pesos.T.dot(self.entradas)
        return self.funcao_de_ativacao(y)

    def calcula_erro(self,y, yd):
        self.erro = yd - y

    def delta(self):
        return self.taxa_aprendizado * self.erro * self.entradas

    def atualiza_pesos(self):
        
            self.pesos =  self.pesos + self.delta()
            self.atualiza_pesos_contador += 1
            print("pesos atualizados: ",self.pesos)

In [ ]:
neuronio = Perceptron(2)
neuronio.treinar(exemplos)

Época:  1
pesos atualizados:  [ -0.10196467 -49.62651343 -51.5349771 ]
pesos atualizados:  [ -0.10196467   6.42252657 -96.2499571 ]
pesos atualizados:  [-1.01964668e-01  1.03697887e+02 -6.85975071e+01]
pesos atualizados:  [-1.01964668e-01  4.40311666e+01 -1.58786617e+02]
pesos atualizados:  [-1.01964668e-01  7.14947566e+01 -1.37930267e+02]
pesos atualizados:  [-1.01964668e-01  1.63724297e+02 -6.40361971e+01]
pesos atualizados:  [-1.01964668e-01  1.52947617e+02 -7.61043271e+01]
pesos atualizados:  [-1.01964668e-01  9.64524966e+01 -1.65997027e+02]
pesos atualizados:  [-1.01964668e-01  1.45913317e+02 -1.17102977e+02]
pesos atualizados:  [-1.01964668e-01  1.17705857e+02 -1.50687097e+02]
pesos atualizados:  [-1.01964668e-01  2.05254247e+02 -8.00879571e+01]
pesos atualizados:  [-1.01964668e-01  1.40560987e+02 -1.61459087e+02]
pesos atualizados:  [-1.01964668e-01  2.10754727e+02 -9.42334071e+01]
pesos atualizados:  [-1.01964668e-01  1.87987707e+02 -1.35667197e+02]
pesos atualizados:  [-1.0196